# `ProteinWorkshop` Tutorial, Part 1 - Training a New Model
![Models](../docs/source/_static/box_models.png)

## Train a new model using the `ProteinWorkshop`

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

Welcome to the tutorial series for the `ProteinWorkshop`! 

In the `ProteinWorkshop`, we implement numerous [featurisation](https://www.proteins.sh/configs/features) schemes, [datasets](https://www.proteins.sh/configs/dataset) for [self-supervised pre-training](https://proteins.sh/quickstart_component/pretrain.html) and [downstream evaluation](https://proteins.sh/quickstart_component/downstream.html), [pre-training](https://proteins.sh/configs/task) tasks, and [auxiliary tasks](https://proteins.sh/configs/task.html#auxiliary-tasks).

[Processed datasets](https://drive.google.com/drive/folders/18i8rLST6ZICTBu6Q67ClT0KqN9AHeqoW?usp=sharing) and [pre-trained weights](https://drive.google.com/drive/folders/1zK1r8FpmGaqV_QwUJuvDacwSL0RW-Vw9?usp=sharing) are made available. Downloading datasets is not required; upon first run all datasets will be downloaded and processed from their respective source.

The `ProteinWorkshop` encompasses several models as well as pre-trained weights for them so that you can readily use them.

In this tutorial, we show you how you can use what is already available in the protein workshop to train and use models for specific tasks. The `ProteinWorkshop` is structured as a very modular package; we will therefore talk about how to change the different parts of it, like the model, training task, dataset, featurization scheme, etc. in this tutorial. 

Besides using all the different options we provide, you can make use of the modular nature of the `ProteinWorkshop` to add your own models, datasets, featurization schemes, and training tasks. We will show you how to do this in the next tutorials.

To train a new model, you can follow the following 3-step procedure:

1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks
2. Validate the designed training config
3. Use the designed config to train a new model

### 1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks

You can switch out any of these for another available option by replacing the corresponding argument's value in `overrides`:

`cfg = hydra.compose("template", overrides=["encoder=schnet", "task=inverse_folding", "dataset=afdb_swissprot_v4", "features=ca_base", "+aux_task=none"], return_hydra_config=True)`

In [ ]:
import os

# Set the environment variable to make only GPU 0 visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Now import the necessary librarie
import torch
# Verify GPUs visible to PyTorch
torch_visible_gpus = torch.cuda.device_count()
print("Visible GPUs for PyTorch:", torch_visible_gpus)

In [2]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")
# print(rel_path)
GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=schnet",
        "encoder.hidden_channels=512", # Number of channels in the hidden layers
        "encoder.out_dim=32", # Output dimension of the model
        "encoder.num_layers=6", # Number of filters used in convolutional layers
        "encoder.num_filters=128", # Number of convolutional layers in the model
        "encoder.num_gaussians=50", # Number of Gaussian functions used for radial filters
        "encoder.cutoff=10.0", # Cutoff distance for interactions
        "encoder.max_num_neighbors=32", # Maximum number of neighboring atoms to consider
        "encoder.readout=add", # Global pooling method to be used
        "encoder.dipole=False",
        "encoder.mean=null",
        "encoder.std=null",
        "encoder.atomref=null",
        "encoder.pretraining=False",

        "task=multiclass_graph_classification",
        "dataset=fold_family",
        "dataset.datamodule.batch_size=32",
        "features=ca_base", 
        "+aux_task=none",
        
        "trainer.max_epochs=150",
        "optimiser=adam",
        "optimiser.optimizer.lr=3e-4",
        "callbacks.early_stopping.patience=10",
        "test=True",
        "scheduler=plateau", # 5 epochs - 0.6 default

        ## for test ONLY
        # "task_name=test",  # here
        # "ckpt_path_test=/home/zhang/Projects/3d/ProteinWorkshop/notebooks/outputs/checkpoints/epoch_016.ckpt", # here
        # "optimizer.weight_decay=0.5"
        "seed=66",
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

[07/23/24 18:46:53] DEBUG    No `.env` file found in project root. Checking for env vars...         ]8;id=920845;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=91892;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/constants.py#22\22]8;;\

                    DEBUG    No env var `DATA_PATH` found. Setting default...                       ]8;id=179255;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=198647;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/constants.py#28\28]8;;\

                    INFO     DATA_PATH:                                                             ]8;id=883477;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=765114;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/constants.py#39\39]8;;\
                             /home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/data                   

[07/23/24 18:46:55] INFO     Hydra initialised at                                                    ]8;id=112939;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=298204;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/config.                

### 2. Validate the designed training config

This is not strictly necessary, but it is a good idea to validate the config before training. This will check that all the arguments you have provided are valid and that the config is complete.

In [3]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

[07/23/24 18:46:56] DEBUG    Requested CPUs: 16. Avialable CPUs (physical): 12. Requested CPU count   ]8;id=707777;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=497236;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py#314\314]8;;\
                             will therefore be set to maximum number ofavailable physical cores.                   
                             NOTE: It is recommended to use N-1cores or less to avoid memory flush                 
                             overheads.                                                                            

                    DEBUG    CUDA available: True                                                     ]8;id=728932;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=178690;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py#249\249]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=980954;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=88882;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py#250\250]8;;\

                    DEBUG    GPU count set to: 1                                                      ]8;id=679229;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=26802;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py#255\255]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=994704;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=241178;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/configs/config.py#165\165]8;;\

In [4]:
print(cfg.keys())
for key in cfg.keys():
    print(key)
    print(cfg[key])

dict_keys(['hydra', 'env', 'dataset', 'features', 'encoder', 'decoder', 'transforms', 'callbacks', 'optimiser', 'scheduler', 'trainer', 'extras', 'metrics', 'task', 'logger', 'hparams', 'name', 'seed', 'num_workers', 'task_name', 'ckpt_path_test', 'test', 'aux_task'])
hydra
{'run': {'dir': '${env.paths.log_dir}/${task_name}/runs/${now:%Y-%m-%d}_${now:%H-%M-%S}'}, 'sweep': {'dir': '${env.paths.log_dir}/${task_name}/multiruns/${now:%Y-%m-%d}_${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from tho

### 3. Use the designed config to train a new model

Now with the config you have designed, you can train a new model. You can also use the `ProteinWorkshop` to evaluate the model on a downstream task.

In [5]:
from proteinworkshop.train import train_model

train_model(cfg)

Seed set to 66


[07/23/24 18:46:57] INFO     Instantiating datamodule:                                                  ]8;id=467335;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=308324;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/train.py#99\99]8;;\
                             <proteinworkshop.datasets.fold_classification.FoldClassificationDataModule            
                             ...                                                                                   

                    INFO     Instantiating callbacks...                                                ]8;id=780438;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=907821;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/train.py#106\106]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.ModelCheckpoint>   ]8;id=134172;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=961687;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.EarlyStopping>     ]8;id=381038;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=763532;file:///home/yang/Yang_ZHANG/Projects/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

...

/home/yang/anaconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: UserWarning:

Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1617. To avoid any 
miscalculations, use `self.log(..., batch_size=batch_size)`.

Metric val/graph_label/accuracy improved. New best score: 0.090
Metric train/loss/total improved. New best score: 4.564
Epoch 0, global step 385: 'val/graph_label/accuracy' reached 0.09025 (best 0.09025), saving model to '/home/yang/Yang_ZHANG/Projects/ProteinWorkshop/notebooks/outputs/checkpoints/epoch_000.ckpt' as top 1


Metric val/graph_label/accuracy improved by 0.004 >= min_delta = 0.0. New best score: 0.094
Metric train/loss/total improved by 1.473 >= min_delta = 0.0. New best score: 3.090
Epoch 1, global step 770: 'val/graph_label/accuracy' reached 0.09398 (best 0.09398), saving model to '/home/yang/Yang_ZHANG/Projects/ProteinWorkshop/notebooks/outputs/checkpoints/epoch_001.ckpt' as top 1


Metric val/graph_label/accuracy improved by 0.058 >= min_delta = 0.0. New best score: 0.152
Metric train/loss/total improved by 0.474 >= min_delta = 0.0. New best score: 2.616
Epoch 2, global step 1155: 'val/graph_label/accuracy' reached 0.15151 (best 0.15151), saving model to '/home/yang/Yang_ZHANG/Projects/ProteinWorkshop/notebooks/outputs/checkpoints/epoch_002.ckpt' as top 1


Metric train/loss/total improved by 0.453 >= min_delta = 0.0. New best score: 2.163
Epoch 3, global step 1540: 'val/graph_label/accuracy' was not in top 1


### 4. Wrapping up

Have any additional questions about using the components provided in the `ProteinWorkshop`? [Create a new issue](https://github.com/a-r-j/ProteinWorkshop/issues/new/choose) on our [GitHub repository](https://github.com/a-r-j/ProteinWorkshop). We would be happy to work with you to leverage the full power of the repository!

/home/yang/anaconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py